In [1]:
# TODO:
# Dont forget about normalized agent state
# Dont forget about put logits to loss, not probablility

In [3]:
!nvidia-smi

Tue Feb 15 17:41:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.08       Driver Version: 472.08       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:15:00.0 Off |                  N/A |
| 30%   36C    P8    17W / 250W |    215MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ... WDDM  | 00000000:21:00.0  On |                  N/A |
| 30%   

In [8]:
import torch
import numpy as np

print(f'Cuda available: {torch.cuda.is_available()}')

#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f'Running on: {device}, Device count: {torch.cuda.device_count()}')

Cuda available: True
Running on: cuda, Device count: 2


### Init dataset

In [11]:
from nuscenes import NuScenes
from custom_prediction.helper import PredictHelper

In [12]:
# This is the path where you stored your copy of the nuScenes dataset.
DATAROOT = 'D:/ThesisProjects/nuScenes_dataset/'

nusc = NuScenes('v1.0-trainval', dataroot=DATAROOT, verbose=False)

In [4]:
from nuscenes.eval.prediction.splits import get_prediction_challenge_split

#train_samples = get_prediction_challenge_split("train", dataroot=DATAROOT)
valid_samples = get_prediction_challenge_split("train_val", dataroot=DATAROOT)

In [5]:
import matplotlib.pyplot as plt

def show_img(img, size=4):
    plt.figure(figsize=(size, size))
    plt.axis('off')
#     img_sorce = img *= 255
    plt.imshow(img)
    plt.show()

### Dataset loading

In [6]:
import numpy as np

NPY_DATAROOT = './npy_dataset/'

In [7]:
train_images = np.load(f'{NPY_DATAROOT}train/train_img_norm.npy')
train_states = np.load(f'{NPY_DATAROOT}train/train_states.npy')
train_labels = np.load(f'{NPY_DATAROOT}train/train_labels.npy')

# train_images = np.load(f'{NPY_DATAROOT}train_img_small.npy')
# train_states = np.load(f'{NPY_DATAROOT}train_states_small.npy')
# train_labels = np.load(f'{NPY_DATAROOT}train_labels_small.npy')

In [8]:
len(train_images), len(train_states), len(train_labels)

(32186, 32186, 32186)

In [9]:
valid_images = np.load(f'{NPY_DATAROOT}validation/validation_img_norm.npy')
valid_states = np.load(f'{NPY_DATAROOT}validation/validation_states.npy')
valid_labels = np.load(f'{NPY_DATAROOT}validation/validation_labels.npy')

# valid_images = np.load(f'{NPY_DATAROOT}valid_img_small.npy')
# valid_states = np.load(f'{NPY_DATAROOT}valid_states_small.npy')
# valid_labels = np.load(f'{NPY_DATAROOT}valid_labels_small.npy')

In [10]:
len(valid_images), len(valid_states), len(valid_labels)

(8560, 8560, 8560)

In [11]:
# Fix error dot : expected both vectors to have same dtype, but found Double and Float
# Because predictions from MTP in type float32

train_labels = train_labels.astype(np.float32)
valid_labels = valid_labels.astype(np.float32)

### Check data

In [14]:
index_to_check = 45

img = train_images[index_to_check]
img = img.reshape((500, 500, 3))

show_img(img)

print(train_states[index_to_check])
print()
print(train_labels[index_to_check])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


ValueError: Unsupported dtype

<Figure size 288x288 with 1 Axes>

[4.3407215  0.01153087 0.03022885]

[ 0.08957963  2.1693192   0.17915927  4.3386383   0.2687389   6.5079575
  0.04116525  9.306195   -0.1864084  12.104433   -0.66802967 15.500806
 -1.1496475  18.896177   -1.6312687  22.292551   -2.11289    25.688923
 -2.5945113  29.085295   -3.1701343  32.774345   -3.7457573  36.463398  ]


In [12]:
def plot_path(trajectory):
    plt.scatter(trajectory[:, 1], -trajectory[:, 0], c='g', s=10)
    # Keep aspect ratio of axis
    plt.axis('equal')
    plt.show()

In [13]:
a = train_labels[index_to_check].reshape(12, 2)
plot_path(a)

NameError: name 'index_to_check' is not defined

### Clear everything before start

In [17]:
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda

# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()                             

#     torch.cuda.empty_cache()

#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)

#     print("GPU Usage after emptying the cache")
#     gpu_usage()

In [18]:
# free_gpu_cache()

### Train

In [12]:
import wandb

wandb.init(
    project='MTP_1.0',
    entity='techtoker',
    config={    
        "epoch": 32,
        "batch_size": 32,
        "num_modes": 4,
        "lr":9e-5,
        "notes": "second try"
    })

config = wandb.config

wandb: Currently logged in as: techtoker (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [13]:
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler

import numpy as np
from typing import List

class NuscenesDataset(Dataset):
    def __init__(self, features: List[str], states: List[str], labels: List[str]):
        
#         features = features.copy()
#         states = states.copy()
        
#         features = features.astype(np.float16)
    
#         # normalization
#         for i in range(len(features)):
#             features[i] = features[i] / 255.0
                    
        max_velocity = 26.0
        min_accel = -26.0
        max_accel = 26.0
        min_heading_cr = -1.5
        max_heading_cr = 1.5
        
        # normalization        
        velocities = [state[0] / max_velocity for state in states]
        accel = [(state[1] + abs(min_accel)) / (abs(min_accel) + max_accel) for state in states]
        heading_cr = [(state[2] + abs(min_heading_cr)) / (abs(min_heading_cr) + max_heading_cr) for state in states]
        
        for i in range(len(states)):
            states[i] = np.array([velocities[i], accel[i], heading_cr[i]])
                
        self.features = features
        self.states = states
        self.labels = labels

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index: int):
        image = self.features[index].reshape((500, 500, 3)) 
        
        #show_img(image)
        
        image = torch.Tensor(image).permute(2, 0, 1)
        
        agent_state_vector = torch.Tensor(self.states[index])
        ground_truth = self.labels[index].reshape((12, 2))

        # Convert to [batch_size, 1, 12, 2]
        # Because loss function need that format
        ground_truth = np.expand_dims(ground_truth, 0)

        return image, agent_state_vector, ground_truth

In [14]:
train_ds = NuscenesDataset(train_images, train_states, train_labels)
train_dl = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)

validation_ds = NuscenesDataset(valid_images, valid_states, valid_labels)
validation_dl = DataLoader(train_ds, batch_size=config.batch_size * 2)

In [15]:
image, state, ground_truth = next(iter(train_dl))

print(image.size())
print(state.size())
print(ground_truth.size())

torch.Size([32, 3, 500, 500])
torch.Size([32, 3])
torch.Size([32, 1, 12, 2])


In [16]:
print(torch.min(image[0]), torch.max(image[0]))
print(state[0])

tensor(0.) tensor(1.0010)
tensor([0.4243, 0.5001, 0.4961])


In [22]:
img = image[0].permute(1, 2, 0).numpy()#.astype(np.float64)

In [23]:
img.shape

(500, 500, 3)

In [24]:
for img in image:
    test = img.permute(1, 2, 0).numpy()
    
    for t1 in test:
        for t2 in t1:
            print(t2)
        
    print()

[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0. 

[0.         0.9213867  0.83154297]
[0.         1.         0.90185547]
[0.         1.         0.90185547]
[0.         1.         0.90185547]
[0.         1.         0.90185547]
[0.         1.         0.90185547]
[0.         1.         0.90185547]
[0.         0.47070312 0.42358398]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.533203

[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[0.45874023 0.         0.05883789]
[0.99609375 0.         0.12939453]
[1.         0.         0.12939453]
[1.         0.         0.12939453]
[1.         0.         0.12939453]
[1.         0.         0.12939453]
[1.         0.         0.12939453]
[1.         0.         0.10192871]
[1.         0.00784302 0.        ]
[1.         0.00784302 0.        ]
[1.         

[0.46655273 0.5332031  0.60009766]
[0.         0.62353516 0.5097656 ]
[0.         1.         0.82373047]
[0.         1.         0.81152344]
[0.         1.         0.81152344]
[0.        1.        0.8154297]
[0.        1.        0.8276367]
[0.         1.         0.83935547]
[0.         1.         0.86279297]
[0.        0.3647461 0.3215332]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.6000

[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.     

...

[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.     

[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.     

[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.46655273 0.5332031  0.60009766]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.

[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.        0.7529297]
[0.        0.   

KeyboardInterrupt: 

In [27]:
show_img(img.astype('uint8'))

AttributeError: 'Tensor' object has no attribute 'astype'

In [17]:
velocities = [state[0] for state in train_states]
accel = [state[1] for state in train_states]
heading_cr = [state[2] for state in train_states]

print(min(velocities), max(velocities))
print(min(accel), max(accel))
print(min(heading_cr), max(heading_cr))

0.0 0.901815166149039
0.02914778124273372 0.9175767206596768
0.17412300484669832 0.9077817172073649


In [29]:
# Output has 50 entries.
# The first 24 are x,y coordinates (in the agent frame) over the next 6 seconds at 2 Hz for the first mode.
# The second 24 are the x,y coordinates for the second mode.
# The last 2 are the logits of the mode probabilities
# mtp(image, state)

In [18]:
from nuscenes.prediction.models.backbone import ResNetBackbone
from nuscenes.prediction.models.mtp import MTP, MTPLoss

backbone = ResNetBackbone('resnet50')
model = MTP(backbone, num_modes=config.num_modes)
model = torch.nn.DataParallel(model)

#model.load_state_dict(torch.load('./Models/norm_MTP_m3_e22_2.56.pth'))
model = model.to(device)

loss_function = MTPLoss(config.num_modes)

In [19]:
from torch.optim import Adam

# Pass to optimizer only params with requires_grad
params_to_update = []

for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

In [20]:
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR, StepLR, ReduceLROnPlateau

optimizer = Adam(params_to_update, lr=config.lr, weight_decay=5e-4)
#scheduler_exp = ExponentialLR(optimizer, gamma=0.9)
scheduler_exp = StepLR(optimizer, step_size=5, gamma=0.9)

scheduler_onPlateau = ReduceLROnPlateau(optimizer, 'min')

In [21]:
import os

# Create folder for all models
dir = os.path.join('./models')
if not os.path.exists(dir):
    os.mkdir(dir)

In [22]:
import nuscenes.eval.prediction.metrics as metrics
from collections import defaultdict
from typing import List, Dict, Any


def GetPrediction(model, instance_token, sample_token, img, state):

    img = img.unsqueeze(0).to(device)
    state = state.unsqueeze(0).to(device)

    predict = model(img, state)
    predict = predict[0]

    trajectories_set = predict[:len(predict) - config.num_modes]
    trajectories_set = trajectories_set.reshape(config.num_modes, 12, 2)
        
    mode_probabilities = predict[-config.num_modes:]

    sorted_logits_indexes = mode_probabilities.argsort(descending=True)
    sorted_trajectories = trajectories_set[sorted_logits_indexes]
    
    mode_probabilities = mode_probabilities.cpu().detach().numpy()
    sorted_trajectories = sorted_trajectories.cpu().detach().numpy()
    
    top_trajectories = sorted_trajectories[:10]
    top_probabilities = mode_probabilities[:10]
    
    # Prediction contains more than 25 modes. 
    prediction = metrics.Prediction(instance_token, sample_token, top_trajectories, top_probabilities)

    return prediction


def GetMetrics(model, valid_ds, valid_samples):
    
    # Define metrics function and container
    metric_functions = [metrics.MinFDEK([1, 5], aggregators=[metrics.RowMean()]),
                        metrics.MinADEK([1, 5, 10], aggregators=[metrics.RowMean()]),
                        metrics.MissRateTopK([5, 10], tolerance=2, aggregators=[metrics.RowMean()])]

    num_predictions = len(valid_ds) # Amount of prediction rows (equals of len of validation)
    metrics_container = {metric.name: np.zeros((num_predictions, metric.shape)) for metric in metric_functions}
    
    # Prediction loop
    for idx in tqdm(range(len(valid_ds))):
        feature, state, ground_truth = valid_ds[idx]
        instance_tkn, sample_tkn = valid_samples[idx].split("_")
                        
        # Make prediction
        prediction = GetPrediction(model, instance_tkn, sample_tkn, feature, state)
        
        # Get ground_truth
        gt = ground_truth.reshape((12, 2))

        # Calculate metrics
        for metric in metric_functions:
            metrics_container[metric.name][idx] = metric(gt, prediction)
    
    # Metrics aggregations
    aggregations: Dict[str, Dict[str, List[float]]] = defaultdict(dict)

    for metric in metric_functions:
        for agg in metric.aggregators:
            aggregations[metric.name][agg.name] = agg(metrics_container[metric.name])

    return aggregations

In [23]:
from tqdm import tqdm
import time

def loss_batch(model, loss_func, img, state_vec, ground_truth, opt=None):
    img = img.to(device)
    state_vec = state_vec.to(device)
    ground_truth = ground_truth.to(device)
        
    predicted_logits = model(img, state_vec)
    loss = loss_func(predicted_logits, ground_truth)

    # For validation optimizer is None, thus we dont perform backprop
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    # Return losses and amount of items
    #print(f"{loss.item()}; {len(img)}")
    return loss.item(), len(img)


def train_step(model, train_dl, loss_func, opt):
    model.train()

    losses = []
    nums = []
    for img, state_vec, gt in tqdm(train_dl):
        loss, num = loss_batch(model, loss_func, img, state_vec, gt, opt)
        losses.append(loss)
        nums.append(num)
        
    # Calculate train loss
    train_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
    return train_loss


def validation_step(model, valid_dl, loss_func):
    tqdm.write("Validation step")
    time.sleep(0.5)
    
    model.eval()

    with torch.no_grad():
        losses = []
        nums = []
        for img, state_vec, gt in tqdm(valid_dl):
            loss, num = loss_batch(model, loss_func, img, state_vec, gt)
            losses.append(loss)
            nums.append(num)
        
        # Calculate validation loss
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        
        scheduler_exp.step()
        scheduler_onPlateau.step(val_loss)
        
    return val_loss


def log_metrics(model, valid_ds, valid_samples, start_epoch_time, best_loss, train_loss, val_loss):
    
    # Calculate metrics on validation set
    tqdm.write("Calculate metrics:")
    time.sleep(0.5)
    
    metricsRes = GetMetrics(model, valid_ds, valid_samples)
    
    minFDE_1 = metricsRes['MinFDEK']['RowMean'][0]
    minFDE_5 = metricsRes['MinFDEK']['RowMean'][1]
    minADEK_1 = metricsRes['MinADEK']['RowMean'][0]
    minADEK_5 = metricsRes['MinADEK']['RowMean'][1]
    minADEK_10 = metricsRes['MinADEK']['RowMean'][2]
    missRateTopK2_5 = metricsRes['MissRateTopK_2']['RowMean'][0]
    missRateTopK2_10 = metricsRes['MissRateTopK_2']['RowMean'][1]    

    wandb.log({'Train loss': train_loss,
               'Val loss': val_loss,
               'Learning_rate': scheduler_exp.get_last_lr()[0],
               'minFDE_1': minFDE_1,
               'minFDE_5': minFDE_5,
               'min_ADEK_1': minADEK_1,
               'minADEK_5': minADEK_5,
               'minADEK_10': minADEK_10,
               'missRateTopK2_5':missRateTopK2_5,
               'missRateTopK2_10':missRateTopK2_10
              })
        
    print(f"Epoch {current_epoch + 1}; Train loss: {train_loss:0.2f}; Val loss: {val_loss:0.2f}; Best VL: {best_loss:0.2f}; Time: {(time.time() - start_epoch_time):0.2f} sec;")

In [24]:
import copy

current_epoch = 0
best_model_wts = copy.deepcopy(model.state_dict())

def fit(epochs, model, loss_func, opt, train_dl, valid_dl, valid_ds, valid_samples):
    global current_epoch
    global best_model_wts
    
    best_loss = 999.0

    for epoch in range(epochs):
        start_epoch_time = time.time()
        tqdm.write('-' * 30)
        tqdm.write(f'Epoch: {epoch + 1}/{epochs}')
        time.sleep(0.5)
        
        train_loss = train_step(model, train_dl, loss_func, opt)        
        val_loss = validation_step(model, valid_dl, loss_func)  
        
        # deep copy the model
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())

        log_metrics(model, valid_ds, valid_samples, start_epoch_time, best_loss, train_loss, val_loss)
            
        torch.save(copy.deepcopy(model.state_dict()), f'./Models/norm_MTP_m4_e{current_epoch}_{best_loss:0.2f}.pth')
        current_epoch += 1

In [25]:
fit(config.epoch, model, loss_function, optimizer, train_dl, validation_dl, validation_ds, valid_samples)

------------------------------
Epoch: 1/32


  0%|                                                                                         | 0/1006 [00:00<?, ?it/s]C:\Program Files\anaconda3\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
100%|██████████████████████████████████████████████████████████████████████████████| 1006/1006 [10:15<00:00,  1.63it/s]


Validation step


100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [04:28<00:00,  1.88it/s]


Calculate metrics:


...

Validation step


100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [04:18<00:00,  1.94it/s]


Calculate metrics:


100%|██████████████████████████████████████████████████████████████████████████████| 8560/8560 [02:44<00:00, 52.14it/s]


Epoch 32; Train loss: 1.91; Val loss: 1.93; Best VL: 1.91; Time: 1024.66 sec;


In [47]:
# optimizer.param_groups[0]['lr'] = 1e-5
# scheduler_exp = StepLR(optimizer, step_size=4, gamma=0.9)